In [1]:
import numpy as np                                                              
import tensorflow as tf                                                         

In [2]:
# Check TensorFlow Version
from distutils.version import LooseVersion
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found.')
else:
    print('Current GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0
Current GPU Device: /device:GPU:0


In [3]:
from tensorflow.contrib.layers import fully_connected

In [4]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [5]:
reset_graph()

### Build Networks

In [6]:
#
# activation functions
#
def leaky_relu(z, name=None):
    return tf.maximum(0.01 * z, z, name=name)


def elu(z, alpha=1):
    return np.where(z < 0, alpha * (np.exp(z) - 1), z)

def selu(z,
         scale=1.0507009873554804934193349852946,
         alpha=1.6732632423543772848170429916717):
    return scale * tf.where(z >= 0.0, z, alpha * tf.nn.elu(z))


In [7]:
# n_inputs = 90  # 45 + 45
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10 # good (1) or bad (0)


X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=selu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=selu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 30
batch_size = 64

### Traing NN

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [9]:
means = mnist.train.images.mean(axis=0, keepdims=True)
stds = mnist.train.images.std(axis=0, keepdims=True) + 1e-10

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch_scaled = (X_batch - means) / stds
            sess.run(training_op, feed_dict={X: X_batch_scaled, y: y_batch})
        if epoch % 5 == 0:
            acc_train = accuracy.eval(feed_dict={X: X_batch_scaled, y: y_batch})
            X_val_scaled = (mnist.validation.images - means) / stds
            acc_test = accuracy.eval(feed_dict={X: X_val_scaled, y: mnist.validation.labels})
            print(epoch, "Batch accuracy:", acc_train, "Validation accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final_selu.ckpt")

(0, 'Batch accuracy:', 0.921875, 'Validation accuracy:', 0.9174)
(5, 'Batch accuracy:', 0.96875, 'Validation accuracy:', 0.95279998)
(10, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.96139997)
(15, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.96700001)
(20, 'Batch accuracy:', 0.96875, 'Validation accuracy:', 0.96960002)
(25, 'Batch accuracy:', 0.96875, 'Validation accuracy:', 0.96899998)
(30, 'Batch accuracy:', 0.984375, 'Validation accuracy:', 0.97079998)
(35, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.96899998)
(40, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.97039998)
(45, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.96939999)
(50, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.96939999)
(55, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.96939999)
(60, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.97000003)
(65, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.96960002)
(70, 'Batch accuracy:', 1.0, 'Validation accuracy:', 0.9702)
(75, 'Batch a

In [10]:
means.shape

(1, 784)

In [11]:
stds.shape

(1, 784)